In [ ]:
from src.dataset.module import AudioDataModule

In [ ]:
def generate_input_sample():
    #TODO: rewrite using cfg.data
    data_module = AudioDataModule(cfg.data)
    data_module.setup()
    dataloader = data_module.train_dataloader()
    
    for batch in dataloader:
        # we're getting batch from train dataloader, that returns a batch of data and labels
        # the first element of batch (batch[0]) is a batch of data (3 dimention for audio)
        # the second one is a batch of labels (batch[1]) -- one dimention tensor
        # So, here to obtain a shape of input data, we have to take the batch of data batch[0]
        # and next take the first element in this batch -- batch[0][0]
        input_dim = batch[0][0].shape
        break

In [ ]:
def check_model_parameters(models, input_sample):
    ...